In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

# Define prompt
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Creamos un model input el cual es el prompt agregando cosa del usuario
model_input = prompt.format_prompt(target_word="behaviour", context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.")

# Define Model
model = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.0)

In [2]:
from langchain.output_parsers import RetryWithErrorOutputParser

# Hacemos un ejemplo de un output mal formateado, aunque aqui deberia ir el output de la llamada a la API
missformatted_output = '{"words": ["conduct", "manner"]}'

# Creamos un parser que se encargara de reintentar el parseo del output en caso de que falle
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)

# Le pasamos el model input que es el prompt con lo que quiere el usuario y la respuesta apra saber si si estaba mal formateado
retry_parser.parse_with_prompt(missformatted_output, model_input)

Suggestions(words=['conduct', 'manner'], reasons=['These words both convey a sense of control and order, which is the opposite of disruptive behaviour in a classroom setting.', 'Both words also imply a level of professionalism and respect, which is important in a classroom environment.'])